<a href="https://www.kaggle.com/alperenkaran/water-potability-caution-with-missing-values?scriptVersionId=88893641" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# Section 1: Main purpose of this notebook

*I have seen many notebooks that imputed missing values using the information from the class labels for the classification task. This should be avoided, because it results in data leakage between train and test sets.*


**Section 2:** I simply look at the dataset, and count the missing values.

**Section 3 Part 1:** I create a binary classification task on an artificial dataset with two random features. Obviously, the accuracy should be around 50% since the random features are not informative about the class labels. When, furthermore, some values of a feature are set to `NaN` and imputed using the wrong way of imputation, the accuracies increased. This should not have happened. This demonstrates why that imputation method is wrong.

**Section 3 Part 2:** Here, I do the same for the Water Potability task, and get around 71%.

**Section 4:** I show how we can avoid data leakage between train and test sets. I do the model selection without using class labels of the test set.

# Section 2: Read the Water Potability data

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
data = pd.read_csv('../input/water-potability/water_potability.csv')
data.sample(3)

,ph,Hardness,Solids,Chloramines,Sulfate,Conductivity,Organic_carbon,Trihalomethanes,Turbidity,Potability
2042,7.669013,205.595635,11579.441693,4.263279,356.136518,407.721613,10.829045,83.243808,4.589513,1
2521,8.240946,245.090825,18001.459186,9.367419,350.614962,344.056871,13.904710,NaN,4.666295,0
472,7.920795,173.357151,13454.634777,6.947895,356.768119,471.368695,16.477840,68.000508,3.374327,0


In [3]:
data.isnull().sum()

ph                 491
Hardness             0
Solids               0
Chloramines          0
Sulfate            781
Conductivity         0
Organic_carbon       0
Trihalomethanes    162
Turbidity            0
Potability           0
dtype: int64

The data has some missing values, and we would like to impute them using some strategies.

# Section 3: What you shouldn't do when imputing missing values

## Part 1: Artificial dataset example

Let me first create an artificial dataset. It has two features consisting of random integers between 0 and 4.

We expect that a machine learning model will have an accuracy about 50%.

In [4]:
random_column1 = np.random.randint(5, size=10000) #just 10000 random integers between 0 and 4
random_column2 = np.random.randint(5, size=10000) #just 10000 random integers between 0 and 4
labels = np.random.randint(2, size=10000) #0 or 1 <- binary label

artificial_data = pd.DataFrame()
artificial_data['random_column1'], artificial_data['random_column2'] = random_column1, random_column2

In [5]:
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier

clf = DecisionTreeClassifier()
cv_result = cross_val_score(clf, artificial_data, labels)
print('Cross validation accuracy:', np.mean(cv_result))

Cross validation accuracy: 0.49160000000000004


Now let's make some values of `random_column1` missing.

In [6]:
for i in range(len(artificial_data)):
    if np.random.randint(4) == 0: #this has a probability 25% of being 0
        artificial_data.iloc[i,0] = np.nan #so, approximately one quarter of the column1 is NaN now.

In [7]:
# impute the missing values with means of labels (SHOULD NOT BE DONE)
artificial_data['random_column1'] = artificial_data['random_column1'].fillna(artificial_data.groupby(labels)['random_column1'].transform('mean'))

clf = DecisionTreeClassifier(random_state=0)
cv_result = cross_val_score(clf, artificial_data, labels)
print('Cross validation accuracy:', np.mean(cv_result))

Cross validation accuracy: 0.6196


**The accuracy increased to 62% !**

This is because we did something we shouldn't have done. We imputed missing values using the labels.

## Part 2: Water potability example

If we impute the missing values using the labels, let's see how much accuracy we get.

In [8]:
data = pd.read_csv('../input/water-potability/water_potability.csv')

In [9]:
# We impute using the labels - SHOULD NOT BE DONE

data['ph'] = data['ph'].fillna(data.groupby(['Potability'])['ph'].transform('mean'))
data['Sulfate']=data['Sulfate'].fillna(data.groupby(['Potability'])['Sulfate'].transform('mean'))
data['Trihalomethanes'] =data['Trihalomethanes'].fillna(data.groupby(['Potability'])['Trihalomethanes'].transform('mean'))

In [10]:
x = data.drop('Potability', axis=1)
y = data['Potability']

In [11]:
clf = DecisionTreeClassifier(random_state=0)
cv_result = cross_val_score(clf, x, y)
print(np.mean(cv_result))

0.7139862223049712


We got around 71% accuracy. This will be an optimistic estimate for the decision tree classifier.

**Note:** I am not saying that 71% accuracy is not achievable. This was just to state that imputing missing values with information using the labels (y values) is wrong, and should be avoided.

# Section 4: The correct way of imputation and analysis

We will

- impute the missing values without using the y values (labels)
- try different classifiers

In [12]:
data = pd.read_csv('../input/water-potability/water_potability.csv')

x = data.drop('Potability', axis=1)
y = data['Potability']

In [13]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=.2, random_state=0)

In [14]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler

#classifiers
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression

In [15]:
classifiers = [DecisionTreeClassifier(random_state=0), RandomForestClassifier(random_state=0), 
               AdaBoostClassifier(random_state=0), SVC(random_state=0), 
               KNeighborsClassifier(), LogisticRegression(random_state=0)]

clf_names = ['DT', 'RF', 'AB', 'SVC', 'KNN', 'LR']

for clfname, clf in zip(clf_names, classifiers):
    pipeline = Pipeline([('scaler',StandardScaler()), ('imputer',SimpleImputer(strategy='mean')), (clfname,clf)])
    cv_result = cross_val_score(pipeline, x_train, y_train)
    print(clfname + ':', np.mean(cv_result))

DT: 0.5893129770992366
RF: 0.65
AB: 0.6057251908396947
SVC: 0.6740458015267177
KNN: 0.6114503816793893
LR: 0.6076335877862596


It looks like SVC is the best classifier among all. Let's tune the hyperparameters.

In [16]:
from sklearn.model_selection import GridSearchCV

pipeline = Pipeline([('scaler',StandardScaler()), ('imputer',SimpleImputer()), ('clf',SVC())])
parameter_grid = {'imputer__strategy':['mean','median'],'clf__C':[.001,.01,.1,.2,.5,1.0,1.5,2.0,5.0], 'clf__kernel':['rbf','linear']}
search = GridSearchCV(pipeline, parameter_grid)
search.fit(x_train, y_train)

print('The best parameters are:', search.best_params_)
print('The best score cross validation score is:', search.best_score_)

The best parameters are: {'clf__C': 1.0, 'clf__kernel': 'rbf', 'imputer__strategy': 'median'}
The best score cross validation score is: 0.6755725190839694


In [17]:
clf = search.best_estimator_

clf.fit(x_train, y_train)
print('The test accuracy is', clf.score(x_test, y_test))

The test accuracy is 0.676829268292683
